In [1]:
import numpy as np
from itertools import cycle
import os

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import datetime as dt
import subprocess
from scipy.ndimage import binary_fill_holes
from importlib import reload
os.chdir("/home/149/ab8992/cosima_regional/development/regional-mom6/")

import regional_mom6 as rm
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 28,Total memory: 112.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33189,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 112.00 GiB
Comm: tcp://127.0.0.1:35009,Total threads: 4
Dashboard: /proxy/36961/status,Memory: 16.00 GiB
Nanny: tcp://127.0.0.1:38229,


# TTide domain

In [3]:

## Choose your coordinates and the name of your experiment


yextent = [-56,-26]
xextent = [142,180]
expt_name = "full-40"

# 
daterange = ["2015-01-01 00:00:00", "2015-12-31 00:00:00"]

## Place where all your input files go
inputdir = f"/g/data/nm03/ab8992/ttide-inputs/{expt_name}/"

## Directory where you'll run the experiment from
rundir = f"/home/149/ab8992/tasman-tides/rundirs/{expt_name}/"

## Directory where raw downloads go before processing
tmpdir = f"/scratch/v45/ab8992/reanalysis_tmp/ttide-20"
# tmpdir = f"/home/149/ab8992/tasman-tides/rundirs/full-20/inputdir/forcing"
# tmpdir = "/home/149/ab8992/tasman-tides/rundirs/mkic/archive/output000/"
## Directory where fre tools are stored
toolpath = "/home/157/ahg157/repos/mom5/src/tools/" ## Compiled tools needed for construction of mask tables
for i in [rundir,tmpdir,inputdir]:
    if not os.path.exists(i):
        subprocess.run(f"mkdir {i} -p",shell=True)

In [3]:
ic = xr.open_dataset(tmpdir + "20150101.ic.nc")


etas = xr.open_dataset(tmpdir + "20150101.surface.nc").zos
ic = xr.merge([ic,etas,etas]).to_netcdf(tmpdir + "/ic_unprocessed")
ic

In [4]:
reload(rm)
expt = rm.experiment(
    xextent,
    yextent,
    daterange,
    0.025,  ## Resolution
    100,      ## #zlayers
    20,       ## dz ratio
    5500,    ## Max depth of ocean
    rundir,
    inputdir,
    toolpath
)



In [23]:
xr.open_dataset(tmpdir + "/ic_unprocessed")

<xarray.Dataset>
Dimensions:  (xh: 760, yh: 795, zl: 100, ny: 795, nxp: 761, nyp: 796, nx: 760)
Coordinates:
  * xh       (xh) float64 142.0 142.1 142.1 142.2 ... 179.8 179.9 179.9 180.0
  * yh       (yh) float64 -55.98 -55.94 -55.91 -55.87 ... -26.09 -26.06 -26.02
  * zl       (zl) float64 8.073 13.49 18.94 ... 5.241e+03 5.345e+03 5.448e+03
    xq       (ny, nxp) float64 ...
    yq       (nyp, nx) float64 ...
Dimensions without coordinates: ny, nxp, nyp, nx
Data variables:
    salt     (zl, yh, xh) float64 ...
    temp     (zl, yh, xh) float64 ...
    u        (zl, ny, nxp) float64 ...
    v        (zl, nyp, nx) float64 ...
    eta_t    (yh, xh) float32 ...
Attributes:
    regrid_method:  bilinear

In [11]:

# a = expt.ocean_forcing(
#     tmpdir,  ## Path to ocean foring files
#     {"time":"time","yh":"yh","yq":"yq","xh":"xh","xq":"xq",
#      "zl":"z_l","eta":"zos","u":"u","v":"v",
#      "tracers":{"salt":"temp","temp":"temp"}},
#     # boundaries = ["south","north","west","east"],
#     gridtype="C"
    # )

a = expt.ocean_forcing(
    tmpdir,  ## Path to ocean foring files
    {"time":"time","y":"latitude","x":"longitude",
     "zl":"depth","eta":"zos","u":"uo","v":"vo",
     "tracers":{"salt":"so","temp":"thetao"}},
    boundaries = ["south","north","west","east"]
    )

INITIAL CONDITIONS
Regridding Velocities...Done.
Regridding Tracers...
Done.
Regridding Free surface...
Saving outputs... done.
BRUSHCUT BOUNDARIES
Processing south...Done.
Processing north...

KeyboardInterrupt: 

Process Dask Worker process (from Nanny):
2023-11-21 20:33:18,441 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
2023-11-21 20:33:18,446 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
2023-11-21 20:33:18,447 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
Process Dask Worker process (from Nanny):
2023-11-21 20:33:18,451 - distributed.nanny - ERROR - Worker process died unexpectedly
2023-11-21 20:33:18,452 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
2023-11-21 20:33:18,453 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
2023-11-21 20:33:18,455 - distributed.nanny - ERROR - Worker process died unexpectedly
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call

In [5]:
expt.bathymetry(
    '/g/data/v45/ab8992/synbath2/SYNBATH_V1.2.nc',
    {"xh":"lon",
     "yh":"lat",
     "elevation":"z"}, ## Again this dictionary just maps mom6 variable names to what they are in your topog.
     minimum_layers = 1,
    #  maketopog=False
    )

Starting to regrid bathymetry. If this process hangs you might be better off calling ESMF directly from a terminal with appropriate computational resources using 

 mpirun ESMF_Regrid -s bathy_original.nc -d topog_raw.nc -m bilinear --src_var elevation --dst_var elevation --netcdf4 --src_regional --dst_regional

This is better for larger domains.


Reading in regridded bathymetry to fix up metadata...done.


In [7]:
expt.FRE_tools((40,40))


NOTE from make_solo_mosaic: there are 0 contacts (align-contact)
congradulation: You have successfully run make_solo_mosaic
MAKE SOLO MOSAIC

CompletedProcess(args='/home/157/ahg157/repos/mom5/src/tools/make_solo_mosaic/make_solo_mosaic --num_tiles 1 --dir . --mosaic_name ocean_mosaic --tile_file hgrid.nc', returncode=0)


cp: './ocean_mosaic.nc' and 'ocean_mosaic.nc' are the same file
cp: './hgrid.nc' and 'hgrid.nc' are the same file


cp ./hgrid.nc hgrid.nc 

NOTE from make_coupler_mosaic: the ocean land/sea mask will be determined by field depth from file topog.nc
mosaic_file is grid_spec.nc

***** Congratulation! You have successfully run make_quick_mosaic
QUICK MOSAIC

CompletedProcess(args='/home/157/ahg157/repos/mom5/src/tools/make_quick_mosaic/make_quick_mosaic --input_mosaic ocean_mosaic.nc --mosaic_name grid_spec --ocean_topog topog.nc', returncode=0)

 ===>NOTE from check_mask: when layout is specified, min_pe and max_pe is set to layout(1)*layout(2)=1600

 ===>NOTE from check_mask: Below is the list of command line arguments.

grid_file = ocean_mosaic.nc
topog_file = topog.nc
min_pe = 1600
max_pe = 1600
layout = 40, 40
halo = 4
sea_level = 0
show_valid_only is not set
nobc = 0

 ===>NOTE from check_mask: End of command line arguments.

 ===>NOTE from check_mask: the grid file is version 2 (mosaic grid) grid which contains field gridfiles

==>NOTE from get_boundary_type: x_boundary_type is solid_walls

==>N

In [8]:
expt.setup_run_directory("",using_payu=True)

Number of CPUs required:  1435


## Add Tides to an Existing Run
Offline I've copied all the inputs from the ttide-20 run. All that this notebook does is use the out-of-the-box NWA scripts to add tidal constituents. I'll then need to go back to the MOM_input file from my old tide test to rectify the new one 

In [4]:
import os
import xarray as xr
os.chdir("/home/149/ab8992/cosima_regional/development/withtides/regional-mom6/")
os.listdir()
import boundary_tides as bt

In [8]:
## VERBATIM COPY OLD CODE AND SEE WHAT HAPPENS

# """
# constituents in TPXO9:
# con =
# "m2  ",  0
# "s2  ",  1
# "n2  ",  2
# "k2  ",  3
# "k1  ",  4
# "o1  ",  5
# "p1  ",  6
# "q1  ",  7
# "mm  ",  8
# "mf  ",  9
# "m4  ", 10
# "mn4 ", 11
# "ms4 ", 12
# "2n2 ", 13
# "s1  "  14
# """
# Use these constituents:
# (MOM6 does not currently support m4, mn4, ms4, 2n2, or s1.)
# constituents = range(0, 10)

constituents = [0]

# Hardcoded for my ttide domain!!! 

horizontal_subset = dict(ny=slice(200,380), nx=slice(850,1100))

# Path to tpxo9.v1 data
tpxo_dir = '/g/data/nm03/TPXO'

# Save output here
output_dir = inputdir + "/forcing"

# Setup NWA boundaries
hgrid = xr.open_dataset(inputdir + '/hgrid.nc')
# segments taken from nwa25_obc.py
segments = [
    bt.Segment(1, 'south', hgrid, output_dir=output_dir),
    bt.Segment(2, 'north', hgrid, output_dir=output_dir),
    bt.Segment(3, 'west', hgrid, output_dir=output_dir),
    bt.Segment(4, 'east', hgrid, output_dir=output_dir)
]
# reload(bt)
bt.write_tpxo(constituents, tpxo_dir, segments, horizontal_subset,"tpxo9.v1.nc")

## Regrid era


In [4]:
expt.setup_era5("/g/data/rt52/era5/single-levels/reanalysis")